Author: Anirudh Iyer
Contact Information: aniiyer@iu.edu
Date: 9/19/2024

Topic: Continual learning through incremental data pipeline on CIFAR-10 using pretrained inceptionv3
    
        Description of the Work: Simple Preprocessing and pretrain for warm starting the model. 

https://dx.doi.org/10.2139/ssrn.4366645 - Designing a Convolutional Neural Network for Image Recognition: A Comparative Study of Different Architectures and Training Techniques

# Warming up the model and data preprocessing

In [1]:
import warnings
warnings.filterwarnings('ignore')
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

from sklearn.cluster import KMeans
from torch.utils.data import Subset

In [2]:
# Define the transformations for CIFAR-10
transform = transforms.Compose([
    transforms.Resize(299),  # Resize to Inception-v3's input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(299, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))  # CIFAR-10 normalization
])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Check a batch of training data
dataiter = iter(trainloader)
images, labels = next(dataiter)
print("Image batch shape: ", images.shape)
print("Labels batch shape: ", labels.shape)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified
Image batch shape:  torch.Size([128, 3, 299, 299])
Labels batch shape:  torch.Size([128])


GoogleNet on ImageNet pretraining

In [3]:
import torchvision.models as models
import torch.nn as nn

# Load Inception-v3 pretrained on ImageNet
inception_v3 = models.inception_v3(pretrained=False)

# Modify the final layer to output 10 classes for CIFAR-10
inception_v3.fc = nn.Linear(inception_v3.fc.in_features, 10)

# Load the saved weights
checkpoint = torch.load('pretrained_inception_v3_e2b100.pth')
inception_v3.load_state_dict(checkpoint)


# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception_v3 = inception_v3.to(device)

# Set the model to training mode
# inception_v3.train()

print(inception_v3)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

Resizing CIFAR-10 to match GoogleNet's input size (299X299) Training on 3 Epochs similar to the paper. 

Inception-v3 provides two outputs:

The primary output (which we want to use for training).
The auxiliary output (used only during training for stabilization, typically ignored in the loss calculation).

In [4]:
# torch.cuda.empty_cache()

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

# Define the optimizer and loss function
optimizer = optim.SGD(inception_v3.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Training loop for 3 epochs
for epoch in range(3):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs, aux_outputs = inception_v3(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}: Loss {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Pretraining')

In [5]:
Save the model's current state (pretrained weights)
torch.save(inception_v3.state_dict(), 'pretrained_inception_v3_e2b100.pth')